In [1]:
#to change accordingly

In [1]:
import pandas as pd
from Sarima import *
import numpy as np
import matplotlib.pyplot as plt
import glob

c:\users\dnl01\appdata\local\programs\python\python38-32\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
def interpolate_method(df,column_name,methode,direction,order,limit):
  df[column_name] = df[column_name].interpolate(method=methode,limit_direction = direction,order=order,limit=limit)
  return df

In [3]:
def clean_csv(df,name,time_variable,quantitative_variable):
  df = df[[time_variable, quantitative_variable]].copy()
  df.columns = ['Date','y']
  df['y'] = (df['y'].apply(pd.to_numeric, errors='coerce')).to_frame()
  df = df.drop_duplicates(subset=['Date'])
  df['Date'] = pd.to_datetime(df.Date, errors = 'coerce')
  df = df.dropna(subset=['Date'])
  df = df.set_index('Date').to_period(name[0])
  df = df.sort_values(by=['Date'])
  df = df.dropna()
  pourcentage = ((len(df) - df['y'].count()) / len(df)) * 100
  if((pourcentage > 2) and (pourcentage < 25) and (2/3 * len(df)) > 400):
    df = interpolate_method(df,'y',"polynomial",None,3,None)
    df=(df-df.min())/(df.max()-df.min())
  elif((pourcentage >= 25) or (2/3 * len(df)) < 600):
    df=(df-df.min())/(df.max()-df.min())
    #print("La base de donnée comporte un grand nombre de données manquantes pour être étudiée, ou n'est pas assez grande.")
    return df, False
  df=(df-df.min())/(df.max()-df.min())
  return df, True

In [4]:
results_300 = pd.DataFrame(columns=['DATA_SET_NAME','Method','MAE','MSE','RMSE'], dtype=object)
results_200 = pd.DataFrame(columns=['DATA_SET_NAME','Method','MAE','MSE','RMSE'], dtype=object)
results_100 = pd.DataFrame(columns=['DATA_SET_NAME','Method','MAE','MSE','RMSE'], dtype=object)

fil = glob.glob("*.csv")
i = 0

for name in fil:
    d = pd.read_csv(name, parse_dates=True, sep=",")
    df = d.copy()
    df, b = clean_csv(df,name,'Date','Value')
    df['y'] = pd.to_numeric(df['y'])
    n = len(df)
    df_300t,df_200t,df_100t = df[:-300]['y'].to_frame(),df[:-200]['y'].to_frame(),df[:-100]['y'].to_frame()
    df_300v,df_200v,df_100v = df[n-300:]['y'].to_frame(),df[n-200:]['y'].to_frame(),df[n-100:]['y'].to_frame()

    try:
        s_300,s_200,s_100 = Sarima(df_300t),Sarima(df_200t),Sarima(df_100t)
        #pred_300, pred_ci_300 = s_300.predict(start=pd.to_datetime(str((df.iloc[n-300:,:]).iloc[0,].name).split(" ")[0]), end=pd.to_datetime(str((df.iloc[n-300:,:]).iloc[-1,].name).split(" ")[0]), dynamic=False, alpha=0.1)
        #pred_200, pred_ci_200 = s_200.predict(start=pd.to_datetime(str((df.iloc[n-200:,:]).iloc[0,].name).split(" ")[0]), end=pd.to_datetime(str((df.iloc[n-200:,:]).iloc[-1,].name).split(" ")[0]), dynamic=False, alpha=0.1)
        #pred_100, pred_ci_100 = s_100.predict(start=pd.to_datetime(str((df.iloc[n-100:,:]).iloc[0,].name).split(" ")[0]), end=pd.to_datetime(str((df.iloc[n-300:,:]).iloc[-1,].name).split(" ")[0]), dynamic=False, alpha=0.1)
        pred_300, pred_ci_300 = s_300.predict(start=df_300v.index[0], end=df_300v.index[-1], dynamic=False, alpha=0.1)
        pred_200, pred_ci_200 = s_200.predict(start=df_200v.index[0], end=df_200v.index[-1], dynamic=False, alpha=0.1)
        pred_100, pred_ci_100 = s_100.predict(start=df_100v.index[0], end=df_100v.index[-1], dynamic=False, alpha=0.1)
        df_new_row_300 = pd.DataFrame(data=np.array([[name,'Sarima',float(s_300.mae(data=df_300v)),float(s_300.mse(data=df_300v)),float(s_300.rmse(data=df_300v))]]), columns=['DATA_SET_NAME','Method','MAE','MSE','RMSE'])
        df_new_row_200 = pd.DataFrame(data=np.array([[name,'Sarima',float(s_200.mae(data=df_200v)),float(s_200.mse(data=df_200v)),float(s_200.rmse(data=df_200v))]]), columns=['DATA_SET_NAME','Method','MAE','MSE','RMSE'])
        df_new_row_100 = pd.DataFrame(data=np.array([[name,'Sarima',float(s_100.mae(data=df_100v)),float(s_100.mse(data=df_100v)),float(s_100.rmse(data=df_100v))]]), columns=['DATA_SET_NAME','Method','MAE','MSE','RMSE'])
        results_300 = pd.concat([results_300,df_new_row_300], ignore_index=True)
        results_200 = pd.concat([results_200,df_new_row_200], ignore_index=True)
        results_100 = pd.concat([results_100,df_new_row_100], ignore_index=True)
        print(f"Processed: {name} {np.array(s_300.prediction).shape}")
        i+=1
    except Exception as e:
        #print(s.season)
        sr = str(e)
        print(f"Error: {name}\n{sr}")
        print(f"300: pred={np.array(s_300.prediction.predicted_mean).shape}, {df_300v.shape}\n200: pred={np.array(s_200.prediction.predicted_mean).shape}, {df_200v.shape}\n100: pred={np.array(s_100.prediction.predicted_mean).shape}, {df_100v.shape}")
#print(results)

print(f"END {i} {len(fil)}")
results_300.to_csv(r'Sarima_results_300.csv', index = False)
results_200.to_csv(r'Sarima_results_200.csv', index = False)
results_100.to_csv(r'Sarima_results_100.csv', index = False)

Processed: M-data_Afghanistan.csv ()
Processed: M-data_Albania.csv ()
Processed: M-data_Algeria.csv ()
Processed: M-data_Andorra.csv ()
Processed: M-data_Antigua_Barbada.csv ()
Processed: M-data_Armenia.csv ()
Processed: M-data_Azerbaijan.csv ()
Processed: M-data_Bahamas.csv ()
Processed: M-data_Baker_Island.csv ()
Processed: M-data_Belgium.csv ()
Processed: M-data_Benin.csv ()
Processed: M-data_Bolivia.csv ()
Processed: M-data_Bosnia_H.csv ()
Processed: M-data_Botswana.csv ()
Processed: M-data_British_Virgin_Island.csv ()
Processed: M-data_Burma.csv ()
Processed: M-data_Cameroon.csv ()
Processed: M-data_Colombia.csv ()
Processed: M-data_Costa_Rica.csv ()
END 19 19
